## Downlaod Data From GEE

In [1]:
import ee
ee.Authenticate()

Enter verification code:  4/1AX4XfWgRhJXBXYUZtZPGWgZlWjZz4lj7nG2NRmg4W07CQZmn_t3RO-mTv5k



Successfully saved authorization token.


In [69]:
ee.Initialize()

In [70]:
from download_utils import Mapdisplay
import folium
import geehydro
import time 
import pandas as pd
import numpy as np

In [71]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");
Country = countries.filter(ee.Filter.eq("country_na","China"));

landsat = l8.filterDate('2019-01-01', '2019-10-31') \
            .filterBounds(Country) 

## this is a cloud masking method in EE
composite = ee.Algorithms.Landsat.simpleComposite(**{
  'collection': landsat,
  'asFloat': True
})
## add layer info
rgbVis = {'bands': ["B4", "B3", "B2"], 'min':0, 'max': 0.3}

In [72]:
Map = folium.Map(location=[40, -100], zoom_start=4)
#Map.setOptions('SATELLITE')
Map.addLayer(composite, rgbVis, "RGB")
Map.centerObject(Country, 4)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

### Download One Country From L8

In [73]:
projection = composite.select('B2').projection().getInfo();

In [74]:
projection

{'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0], 'type': 'Projection'}

In [75]:
f_name = "L8_Country"
scale = 30 ## usually 30 

In [104]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': composite.toFloat(),
    'description': f_name,
    'folder':'GEE_Resources_Raw',   ## it will create a folder an dump the image there 
    'scale': scale,
    'crs': projection['crs'],
    'crsTransform': projection['transform'],
    'region': Japan.geometry(),
    'fileFormat': 'GeoTIFF',
    'maxPixels':1e13,
})
task.start()

while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(1200)
    
print(task.status())

Polling for task (id: S6MFACGLLLPV3BPB33PE6P2X).
Polling for task (id: S6MFACGLLLPV3BPB33PE6P2X).
Polling for task (id: S6MFACGLLLPV3BPB33PE6P2X).
Polling for task (id: S6MFACGLLLPV3BPB33PE6P2X).
Polling for task (id: S6MFACGLLLPV3BPB33PE6P2X).
Polling for task (id: S6MFACGLLLPV3BPB33PE6P2X).
{'state': 'COMPLETED', 'description': 'L8_Country', 'creation_timestamp_ms': 1651465066093, 'update_timestamp_ms': 1651471489590, 'start_timestamp_ms': 1651465082570, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1IlG_KVAL6C3W9xrswbswFBRKjve2YXSu'], 'attempt': 1, 'id': 'S6MFACGLLLPV3BPB33PE6P2X', 'name': 'projects/earthengine-legacy/operations/S6MFACGLLLPV3BPB33PE6P2X'}


### Download From Modise

In [181]:
locations_filepath="data/yield_data.csv"
collection_id="MODIS/006/MCD43A4"

In [182]:
imgcoll = ee.ImageCollection(collection_id) \
    .filterBounds(ee.Geometry.Rectangle(-106.5, 50, -64, 23)) \
    .filterDate("2002-12-31", "2005-8-4")

In [183]:
# Get the number of images.
count = imgcoll.size()
print('Count: ', str(count.getInfo())+'\n')

Count:  947



In [184]:
def _append_im_band(current, previous):
    # Transforms an Image Collection with 1 band per Image into a single Image with items as bands
    # Author: Jamie Vleeshouwer

    # Rename the band
    previous = ee.Image(previous)
    current = current.select([0, 1, 2, 3, 4, 5, 6])
    # Append it to the result (Note: only return current item on first element/iteration)
    return ee.Algorithms.If(
        ee.Algorithms.IsEqual(previous, None),
        current,
        previous.addBands(ee.Image(current)),
    )

In [185]:
img = imgcoll.iterate(_append_im_band)
img = ee.Image(img)

In [186]:
print(len(img.getInfo()['bands']))
print(count.getInfo()*7)

6629
6629


In [187]:
region = ee.FeatureCollection("TIGER/2018/Counties")

In [188]:
# turn the strings into numbers, see
# https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_Counties
def county_to_int(feature):
    return feature.set("COUNTYFP", ee.Number.parse(feature.get("COUNTYFP")))

def state_to_int(feature):
    return feature.set("STATEFP", ee.Number.parse(feature.get("STATEFP")))

region = region.map(county_to_int)
region = region.map(state_to_int)

In [189]:
df = pd.read_csv(locations_filepath)

In [190]:
df.head()

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,SURVEY,2016,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,DALLAS,...,NaN,NaN,0,NaN,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,32.4,NaN
1,SURVEY,2016,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,ELMORE,...,NaN,NaN,0,NaN,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,20.3,NaN
2,SURVEY,2016,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,HALE,...,NaN,NaN,0,NaN,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,23.0,NaN
3,SURVEY,2016,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,LOWNDES,...,NaN,NaN,0,NaN,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,29.3,NaN
4,SURVEY,2016,YEAR,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,MARENGO,...,NaN,NaN,0,NaN,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",TOTAL,NOT SPECIFIED,30.2,NaN


In [191]:
x = np.unique(df[["State ANSI", "County ANSI"]].values, axis=0)
x.shape

(3522, 2)

In [192]:
state_id, county_id = x[10]

In [193]:
fname = "{}_{}".format(int(state_id), int(county_id))

In [194]:
fname

'1_35'

In [195]:
file_region = region.filterMetadata(
    "COUNTYFP", "equals", int(county_id)
).filterMetadata("STATEFP", "equals", int(state_id))

In [196]:
file_region = ee.Feature(file_region.first())
processed_img = img.clip(file_region)
#file_region = None

In [198]:
def _export_one_image(img, folder, name, region, scale, crs):
    # export one image from Earth Engine to Google Drive
    # Author: Jiaxuan You, https://github.com/JiaxuanYou
    print("Exporting to {}/{}".format(folder,name))
    task = ee.batch.Export.image.toDrive(**{
                'image': img,
                'description': name,
                'folder':folder,   ## it will create a folder an dump the image there 
                'scale': scale,
                'crs': crs,
                'region': region,
                'fileFormat': 'GeoTIFF',
                'maxPixels':1e13,
            })
    task.start()
    while task.status()["state"] == "RUNNING":
        print("Running...")
        # Perhaps task.cancel() at some point.
        time.sleep(10)

    print("Done: {}".format(task.status()))

In [199]:
folder_name = 'GEE_Resources_Raw'
print(fname)
coordinate_system="EPSG:4326",
scale=500

1_35


In [202]:
projection = processed_img.select(0).projection().getInfo()

In [ ]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': processed_img.toFloat(),
    'description': fname,
    'folder':folder_name,   ## it will create a folder an dump the image there 
    'scale': scale,
    'crs': projection['crs'],
    'crsTransform': projection['transform'],
    'region': file_region.geometry(),
    'fileFormat': 'GeoTIFF',
    'maxPixels':1e13,
})
task.start()

while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(120)
    
print(task.status())

Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id: 2PN646R55ZUIXWV2RDE2OCX4).
Polling for task (id

In [163]:
int(county_id)

35

In [164]:
dataset = ee.FeatureCollection('TIGER/2018/Counties');
visParams = {
  'palette': ['purple', 'blue', 'green', 'yellow', 'orange', 'red'],
  'min': 0,
  'max': 50,
  'opacity': 0.8,
}

## Turn the strings into numbers
dataset = dataset.map(lambda f: f.set('STATEFP', ee.Number.parse(f.get('STATEFP'))))
dataset = dataset.map(lambda f: f.set('COUNTYFP', ee.Number.parse(f.get('COUNTYFP'))))


file_region = dataset.filterMetadata("STATEFP", "equals", int(state_id)) \
            .filterMetadata(
                "COUNTYFP", "equals", int(county_id)
            )

image = ee.Image().float().paint(file_region)
                      
countyOutlines = ee.Image().float().paint(**{
  'featureCollection': dataset,
  'color': 'black',
  'width': 1
});


Map = folium.Map(location=[40, -100], zoom_start=4)
#Map.setOptions('SATELLITE')
#Map.setCenter(-99.844, 37.649, 5)
Map.centerObject(file_region, 8)
Map.addLayer(image, visParams, 'TIGER/2018/Counties')
Map.addLayer(countyOutlines, {}, 'county outlines')
#Map.addLayer(dataset, {}, 'for Inspector', false)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

In [177]:
one_img = imgcoll.max().clipToCollection(dataset).select(['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band4',
  'Nadir_Reflectance_Band3'])

In [180]:
Map = folium.Map(location=[40, -100], zoom_start=4)
#Map.setOptions('SATELLITE')
#Map.setCenter(-99.844, 37.649, 5)
Map.centerObject(file_region, 5)
Map.addLayer(one_img, {
  'min': 0.0,
  'max': 4000.0,
  'gamma': 1.4,
}, 'TIGER/2018/Counties')
Map.addLayer(countyOutlines, {}, 'county outlines')
#Map.addLayer(dataset, {}, 'for Inspector', false)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map